In [1]:
import pulp
from pulp import*
import pandas as pd

## Sequential optimize

### 5 different formulation
#### 1.  No goalie opponent constraint

In [2]:
k=100
cov_ub=5

In [3]:
def optimization_form_1 (k,I:list,cov_ub:float,Y:dict ):
    # this formulation doesn't have goalie_opponent constraint
    '''
    I: we solve for ith entry sequentially; for i in I
    cov_ub: uperbound of pairwise cov
    '''
    
    prob = pulp.LpProblem("Iter"+str(I[0]), pulp.LpMaximize)
    x = pulp.LpVariable.dicts("X",((i, j) for i in range(1,k) for j in all_player_list), 0 , 1,cat='Integer')
    # t[i,l] indicator: for a lineup i, if there are any player comes from team l
    ## this is for constraint |team| >=3 
    t = pulp.LpVariable.dicts("T",((i, l) for i in range(1,k) for l in team_list), 0 , 1,cat='Integer')

    # v[i,l] complete line indicator (a line has 3player from same line)
    v = pulp.LpVariable.dicts("V",((i, line) for i in range(1,k) for line in line_list), 0 , 1,cat='Integer')

    # partial line indicator
    w = pulp.LpVariable.dicts("W",((i, line) for i in range(1,k) for line in line_list), 0 , 1,cat='Integer')
    #init a table YY to document previously calculated result for 1....i-1 entry 
    #init a table YY to document previously calculated result for 1....i-1 entry    
    #obj
    prob += lpSum(x[(I[0],j)]* mu[j] for j in all_player_list) 
    #constraints
    # sum(salary)<50000 for each lineup
    for i in I:
        prob += lpSum(x[(i,j)] * salary_dict[j] for j in all_player_list) <=50000
    # each lineup has 9 player
    for i in I:
        prob += lpSum(x[(i,j)] for j in all_player_list) == 9
    # for a lineup  2<=center<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in c_list) >= 2
        prob += lpSum(x[(i,j)] for j in c_list) <= 3
    # similar for W,D,G
    #  3<=wing<=4
    for i in I:
        prob += lpSum(x[(i,j)] for j in w_list) >= 3
        prob += lpSum(x[(i,j)] for j in w_list) <= 4
    # 2<=Defense<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in d_list) >= 2
        prob += lpSum(x[(i,j)] for j in d_list) <= 3
    # goalie ==1
    for i in I:
        prob += lpSum(x[(i,j)] for j in g_list) == 1

    #4.2 team constrant : 
    ## players for each lineup must comefrom at least 3 teams.
    for i in I:
        for l in team_list:
            prob += t[(i,l)] <= lpSum(x[(i,j)] for j in team_dict[l]) #for lineup i team l,t[i,l] = 1  => exist j from l in i
    for i in I:       
        prob += lpSum(t[(i,l)] for l in team_list) >= 3 # at least 3 teams in each lineup
                          

    #line stacking
    ## force each lineup has one complete line, two partial line
    for i in I:
        for line in line_list:
            prob += 3 * v[(i,line)] <= lpSum(x[(i,j)] for j in line_dict[line])  #at least 3 player to build a complete line
        prob += lpSum(v[i,line] for line in line_list) >= 1 #at least one complet line
    ## at least two player from two distinct line
    for i in I:
        for line in line_list:
            prob += 2 * w[(i,line)] <= lpSum(x[(i,j)] for j in line_dict[line])  #at least 3 player to build a complete line
        prob += lpSum(w[i,line] for line in line_list) >= 2 #at least one complet line

    # defense stacking
    ## exclude all player that are not in team's first power line
    for i in I:
        for j in non_first_powerline_list:
            prob+= x[(i,j)] == 0
   
    #4.4 overlap
    # from previously calculated entry we could get a table Y[i,j], 
    # relax overlap constraint for first entry
    if I[0] == 1:
        prob.solve()
        
        status = prob.status
        if status !=1:
            print(I)
            print(status)
        varsdict = {}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                entry = int(key.split(",_")[0][-1])
                player = str(key.split(",_")[1][1:-2])
                Y[(entry,player)] = value
        return(status,varsdict,Y)
    if I[0]>=2:
        for i in range(1,I[0]):
            prob += lpSum(Y[(i,j)]*x[(I[0],j)] for j in all_player_list) <= cov_ub
        prob.solve()
        status = prob.status
        if prob.status != 1:
            print(I)
            print(status)
        varsdict ={}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                if I[0]>=10:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])
                else:
                    entry = int(key.split(",_")[0][-1])
                player = key.split(",_")[1][1:-2]
                Y[(entry,player)] = value
        return(status,varsdict,Y)            

#### formulation2 no goalie stacking, two defense man

In [4]:
def optimization_form_2 (k,I:list,cov_ub:float,Y:dict ):
    '''
    I: we solve for ith entry sequentially; for i in I
    cov_ub: uperbound of pairwise cov
    '''

    prob = pulp.LpProblem("Iter"+str(I[0]), pulp.LpMaximize)
    x = pulp.LpVariable.dicts("X",((i, j) for i in range(1,k) for j in all_player_list), 0 , 1,cat='Integer')
    # t[i,l] indicator: for a lineup i, if there are any player comes from team l
    ## this is for constraint |team| >=3 
    t = pulp.LpVariable.dicts("T",((i, l) for i in range(1,k) for l in team_list), 0 , 1,cat='Integer')

    # v[i,l] complete line indicator (a line has 3player from same line)
    v = pulp.LpVariable.dicts("V",((i, line) for i in range(1,k) for line in line_list), 0 , 1,cat='Integer')

    # partial line indicator
    w = pulp.LpVariable.dicts("W",((i, line) for i in range(1,k) for line in line_list), 0 , 1,cat='Integer')
    #init a table YY to document previously calculated result for 1....i-1 entry 
    #obj
    prob += lpSum(x[(I[0],j)]* mu[j] for j in all_player_list) 
    #constraints
    # sum(salary)<50000 for each lineup
    for i in I:
        prob += lpSum(x[(i,j)] * salary_dict[j] for j in all_player_list) <=50000
    # each lineup has 9 player
    for i in I:
        prob += lpSum(x[(i,j)] for j in all_player_list) == 9
    # for a lineup  2<=center<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in c_list) >= 2
        prob += lpSum(x[(i,j)] for j in c_list) <= 3
    # similar for W,D,G
    #  3<=wing<=4
    for i in I:
        prob += lpSum(x[(i,j)] for j in w_list) >= 3
        prob += lpSum(x[(i,j)] for j in w_list) <= 4
    # 2<=Defense<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in d_list) == 2
    # goalie ==1
    for i in I:
        prob += lpSum(x[(i,j)] for j in g_list) == 1

    #4.2 team constrant : 
    ## players for each lineup must comefrom at least 3 teams.
    for i in I:
        for l in team_list:
            prob += t[(i,l)] <= lpSum(x[(i,j)] for j in team_dict[l]) #for lineup i team l,t[i,l] = 1  => exist j from l in i
    for i in I:       
        prob += lpSum(t[(i,l)] for l in team_list) >= 3 # at least 3 teams in each lineup


    #line stacking
    ## force each lineup has one complete line, two partial line
    for i in I:
        for line in line_list:
            prob += 3 * v[(i,line)] <= lpSum(x[(i,j)] for j in line_dict[line])  #at least 3 player to build a complete line
        prob += lpSum(v[i,line] for line in line_list) >= 1 #at least one complet line
    ## at least two player from two distinct line
    for i in I:
        for line in line_list:
            prob += 2 * w[(i,line)] <= lpSum(x[(i,j)] for j in line_dict[line])  #at least 3 player to build a complete line
        prob += lpSum(w[i,line] for line in line_list) >= 2 #at least one complet line

    # defense stacking
    ## exclude all player that are not in team's first power line
    for i in I:
        for j in non_first_powerline_list:
            prob+= x[(i,j)] == 0
   
    #4.4 overlap
    # from previously calculated entry we could get a table Y[i,j], 
    # relax overlap constraint for first entry
    if I[0] == 1:
        prob.solve()
        
        status = prob.status
        if status !=1:
            print(I)
            print(status)
        varsdict = {}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                entry = int(key.split(",_")[0][-1])
                player = str(key.split(",_")[1][1:-2])
                Y[(entry,player)] = value
        return(status,varsdict,Y)
    if I[0]>=2:
        for i in range(1,I[0]):
            prob += lpSum(Y[(i,j)]*x[(I[0],j)] for j in all_player_list) <= cov_ub
        prob.solve()
        status = prob.status
        if prob.status != 1:
            print(I)
            print(status)
        varsdict ={}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                if I[0]>=10:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])
                else:
                    entry = int(key.split(",_")[0][-1])
                player = key.split(",_")[1][1:-2]
                Y[(entry,player)] = value
        return(status,varsdict,Y)            

#### formulation3 baseline formulation (all constraints)

In [5]:
def optimization_form_3 (k,I:list,cov_ub:float,Y:dict ):
    '''
    I: we solve for ith entry sequentially; for i in I
    cov_ub: uperbound of pairwise cov
    '''

    prob = pulp.LpProblem("Iter"+str(I[0]), pulp.LpMaximize)
    x = pulp.LpVariable.dicts("X",((i, j) for i in range(1,k) for j in all_player_list), 0 , 1,cat='Integer')
    # t[i,l] indicator: for a lineup i, if there are any player comes from team l
    ## this is for constraint |team| >=3 
    t = pulp.LpVariable.dicts("T",((i, l) for i in range(1,k) for l in team_list), 0 , 1,cat='Integer')

    # v[i,l] complete line indicator (a line has 3player from same line)
    v = pulp.LpVariable.dicts("V",((i, line) for i in range(1,k) for line in line_list), 0 , 1,cat='Integer')

    # partial line indicator
    w = pulp.LpVariable.dicts("W",((i, line) for i in range(1,k) for line in line_list), 0 , 1,cat='Integer')
    #init a table YY to document previously calculated result for 1....i-1 entry 
    #init a table YY to document previously calculated result for 1....i-1 entry    
    #obj
    prob += lpSum(x[(I[0],j)]* mu[j] for j in all_player_list) 
    #constraints
    # sum(salary)<50000 for each lineup
    for i in I:
        prob += lpSum(x[(i,j)] * salary_dict[j] for j in all_player_list) <=50000
    # each lineup has 9 player
    for i in I:
        prob += lpSum(x[(i,j)] for j in all_player_list) == 9
    # for a lineup  2<=center<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in c_list) >= 2
        prob += lpSum(x[(i,j)] for j in c_list) <= 3
    # similar for W,D,G
    #  3<=wing<=4
    for i in I:
        prob += lpSum(x[(i,j)] for j in w_list) >= 3
        prob += lpSum(x[(i,j)] for j in w_list) <= 4
    # 2<=Defense<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in d_list) >= 2
        prob += lpSum(x[(i,j)] for j in d_list) <= 3
    # goalie ==1
    for i in I:
        prob += lpSum(x[(i,j)] for j in g_list) == 1

    #4.2 team constrant : 
    ## players for each lineup must comefrom at least 3 teams.
    for i in I:
        for l in team_list:
            prob += t[(i,l)] <= lpSum(x[(i,j)] for j in team_dict[l]) #for lineup i team l,t[i,l] = 1  => exist j from l in i
    for i in I:       
        prob += lpSum(t[(i,l)] for l in team_list) >= 3 # at least 3 teams in each lineup
                          
    #4.3
    #goalie stacking
    ## force the goalie variable xik to be zero if the lineup has any skater opposing goalie k
    for i in I:    
        for j in g_list:
            prob += lpSum(x[(i,l)] for l in skater_team_dict[g_opponent_dict[j]]) <= 6*(1-x[i,j])  
            #print(j)
            #print(skater_team_dict[g_opponent_dict[j]])
            #print(lpSum(x[(i,l)] for l in skater_team_dict[g_opponent_dict[j]]) <= 6*(1-x[i,j]))

    #line stacking
    ## force each lineup has one complete line, two partial line
    for i in I:
        for line in line_list:
            prob += 3 * v[(i,line)] <= lpSum(x[(i,j)] for j in line_dict[line])  #at least 3 player to build a complete line
        prob += lpSum(v[i,line] for line in line_list) >= 1 #at least one complet line
    ## at least two player from two distinct line
    for i in I:
        for line in line_list:
            prob += 2 * w[(i,line)] <= lpSum(x[(i,j)] for j in line_dict[line])  #at least 3 player to build a complete line
        prob += lpSum(w[i,line] for line in line_list) >= 2 #at least one complet line

    # defense stacking
    ## exclude all player that are not in team's first power line
    for i in I:
        for j in non_first_powerline_list:
            prob+= x[(i,j)] == 0
   
    #4.4 overlap
    # from previously calculated entry we could get a table Y[i,j], 
    # relax overlap constraint for first entry
    if I[0] == 1:
        prob.solve()
        
        status = prob.status
        if status !=1:
            print(I)
            print(status)
        varsdict = {}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                entry = int(key.split(",_")[0][-1])
                player = str(key.split(",_")[1][1:-2])
                Y[(entry,player)] = value
        return(status,varsdict,Y)
    if I[0]>=2:
        for i in range(1,I[0]):
            prob += lpSum(Y[(i,j)]*x[(I[0],j)] for j in all_player_list) <= cov_ub
        prob.solve()
        status = prob.status
        if prob.status != 1:
            print(I)
            print(status)
        varsdict ={}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                if I[0]>=10:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])
                else:
                    entry = int(key.split(",_")[0][-1])
                player = key.split(",_")[1][1:-2]
                Y[(entry,player)] = value
        return(status,varsdict,Y)            

#### formulation4 exact 3 team all other constraint

In [6]:
def optimization_form_4 (k,I:list,cov_ub:float,Y:dict ):
    '''
    I: we solve for ith entry sequentially; for i in I
    cov_ub: uperbound of pairwise cov
    '''
    prob = pulp.LpProblem(("opt"), pulp.LpMaximize)
    x = pulp.LpVariable.dicts("X",((i, j) for i in range(1,k) for j in all_player_list), 0 , 1,cat='Integer')
    # t[i,l] indicator: for a lineup i, if there are any player comes from team l
    ## this is for constraint |team| >=3 
    t = pulp.LpVariable.dicts("T",((i, l) for i in range(1,k) for l in team_list), 0 , 1,cat='Integer')

    # v[i,l] complete line indicator (a line has 3player from same line)
    v = pulp.LpVariable.dicts("V",((i, team, line) for i in range(1,k) for team in team_list for line in line_list), 0 , 1,cat='Integer')

    # partial line indicator
    w = pulp.LpVariable.dicts("W",((i, team, line) for i in range(1,k) for team in team_list for line in line_list), 0 , 1,cat='Integer')
    #init a table YY to document previously calculated result for 1....i-1 entry    
    
    #obj
    prob += lpSum(x[(I[0],j)]* mu[j] for j in all_player_list) 
    #constraints
    # sum(salary)<50000 for each lineup
    for i in I:
        prob += lpSum(x[(i,j)] * salary_dict[j] for j in all_player_list) <=50000
    # each lineup has 9 player
    for i in I:
        prob += lpSum(x[(i,j)] for j in all_player_list) == 9
    # for a lineup  2<=center<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in c_list) >= 2
        prob += lpSum(x[(i,j)] for j in c_list) <= 3
    # similar for W,D,G
    #  3<=wing<=4
    for i in I:
        prob += lpSum(x[(i,j)] for j in w_list) >= 3
        prob += lpSum(x[(i,j)] for j in w_list) <= 4
    # 2<=Defense<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in d_list) >= 2
        prob += lpSum(x[(i,j)] for j in d_list) <= 3
    # goalie ==1
    for i in I:
        prob += lpSum(x[(i,j)] for j in g_list) == 1

    #4.2 team constrant : 
    ## players for each lineup must come from exact 3 team
    
    for i in I:
        for l in team_list:
            prob += 6*t[(i,l)] >= lpSum(x[(i,j)] for j in team_dict[l])
            prob += t[(i,l)] <= lpSum(x[(i,j)] for j in team_dict[l])
    for i in I:       
        prob += lpSum(t[(i,l)] for l in team_list) == 3 # at least 3 teams in each lineup
                          
    #4.3
    #goalie stacking
    ## force the goalie variable xik to be zero if the lineup has any skater opposing goalie k
    for i in I:    
        for j in g_list:
            prob += lpSum(x[(i,l)] for l in skater_team_dict[g_opponent_dict[j]]) <= 6*(1-x[i,j])  
            #print(j)
            #print(skater_team_dict[g_opponent_dict[j]])
            #print(lpSum(x[(i,l)] for l in skater_team_dict[g_opponent_dict[j]]) <= 6*(1-x[i,j]))

    #line stacking
    ## force each lineup has one complete line, two partial line
    for i in I:
        for team in team_list: 
            for line in line_list:
                prob += 3 * v[(i,team,line)] <= lpSum(x[(i,j)] for j in team_line_dict[(team,line)])  #at least 3 player to build a complete line
        prob += lpSum(v[(i,team,line)] for line in line_list for team in team_list) >= 1 #at least one complet line
        #print(lpSum(v[(i,team,line)] for line in line_list for team in team_list) >= 1)
    ## at least two player from two distinct line
    for i in I:
        for team in team_list: 
            for line in line_list:
                prob += 2 * w[(i,team,line)] <= lpSum(x[(i,j)] for j in team_line_dict[(team,line)])  #at least 3 player to build a complete line
        prob += lpSum(w[(i,team,line)] for line in line_list for team in team_list) >= 2 #at least one complet line
    ## exclude all defense player that are not in team's first power line
    for i in I:
        for j in non_first_powerline_list:
            prob+= x[(i,j)] == 0

    #4.4 overlap
    # from previously calculated entry we could get a table Y[i,j], 
    # relax overlap constraint for first entry
    if I[0] == 1:
        prob.solve()
        
        status = prob.status
        if status !=1:
            print(I)
            print(status)
        varsdict = {}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                entry = int(key.split(",_")[0][-1])
                player = str(key.split(",_")[1][1:-2])
                Y[(entry,player)] = value
        return(status,varsdict,Y)
    if I[0]>=2:
        for i in range(1,I[0]):
            prob += lpSum(Y[(i,j)]*x[(I[0],j)] for j in all_player_list) <= cov_ub
            #cov_ub+=0.1
        prob.solve(GLPK())
        status = prob.status
        if prob.status != 1:
            print(I)
            print(status)
        varsdict ={}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                if I[0]>=100:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])+100*int(key.split(",_")[0][-3])
                elif I[0]>=10:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])
                else:
                    entry = int(key.split(",_")[0][-1])
                player = key.split(",_")[1][1:-2]
                Y[(entry,player)] = value
        return(status,varsdict,Y)            

#### formulation_5 3team no goalie stacking

In [7]:
def optimization_form_5 (k,I:list,cov_ub:float,Y:dict ):
    '''
    I: we solve for ith entry sequentially; for i in I
    cov_ub: uperbound of pairwise cov
    '''
    prob = pulp.LpProblem(("opt"), pulp.LpMaximize)
    x = pulp.LpVariable.dicts("X",((i, j) for i in range(1,k) for j in all_player_list), 0 , 1,cat='Integer')
    # t[i,l] indicator: for a lineup i, if there are any player comes from team l
    ## this is for constraint |team| >=3 
    t = pulp.LpVariable.dicts("T",((i, l) for i in range(1,k) for l in team_list), 0 , 1,cat='Integer')

    # v[i,l] complete line indicator (a line has 3player from same line)
    v = pulp.LpVariable.dicts("V",((i, team, line) for i in range(1,k) for team in team_list for line in line_list), 0 , 1,cat='Integer')

    # partial line indicator
    w = pulp.LpVariable.dicts("W",((i, team, line) for i in range(1,k) for team in team_list for line in line_list), 0 , 1,cat='Integer')
    #init a table YY to document previously calculated result for 1....i-1 entry    
    
    #obj
    prob += lpSum(x[(I[0],j)]* mu[j] for j in all_player_list) 
    #constraints
    # sum(salary)<50000 for each lineup
    for i in I:
        prob += lpSum(x[(i,j)] * salary_dict[j] for j in all_player_list) <=50000
    # each lineup has 9 player
    for i in I:
        prob += lpSum(x[(i,j)] for j in all_player_list) == 9
    # for a lineup  2<=center<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in c_list) >= 2
        prob += lpSum(x[(i,j)] for j in c_list) <= 3
    # similar for W,D,G
    #  3<=wing<=4
    for i in I:
        prob += lpSum(x[(i,j)] for j in w_list) >= 3
        prob += lpSum(x[(i,j)] for j in w_list) <= 4
    # 2<=Defense<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in d_list) >= 2
        prob += lpSum(x[(i,j)] for j in d_list) <= 3
    # goalie ==1
    for i in I:
        prob += lpSum(x[(i,j)] for j in g_list) == 1

    #4.2 team constrant : 
    ## players for each lineup must come from exact 3 team
    ### Notice these three only count for 8 skaters
    for i in I:
        for l in team_list:
            prob += 6*t[(i,l)] >= lpSum(x[(i,j)] for j in skater_team_dict[l])
            prob += t[(i,l)] <= lpSum(x[(i,j)] for j in skater_team_dict[l])
    for i in I:       
        prob += lpSum(t[(i,l)] for l in team_list) == 3 # at least 3 teams in each lineup
                          
    #4.3
    # no goalie stacking


    #line stacking
    ## force each lineup has one complete line, two partial line
    for i in I:
        for team in team_list: 
            for line in line_list:
                prob += 3 * v[(i,team,line)] <= lpSum(x[(i,j)] for j in team_line_dict[(team,line)])  #at least 3 player to build a complete line
        prob += lpSum(v[(i,team,line)] for line in line_list for team in team_list) >= 1 #at least one complet line
        #print(lpSum(v[(i,team,line)] for line in line_list for team in team_list) >= 1)
    ## at least two player from two distinct line
    for i in I:
        for team in team_list: 
            for line in line_list:
                prob += 2 * w[(i,team,line)] <= lpSum(x[(i,j)] for j in team_line_dict[(team,line)])  #at least 3 player to build a complete line
        prob += lpSum(w[(i,team,line)] for line in line_list for team in team_list) >= 2 #at least one complet line
    ## exclude all player that are not in team's first power line
    for i in I:
        for j in non_first_powerline_list:
            prob+= x[(i,j)] == 0

    #4.4 overlap
    # from previously calculated entry we could get a table Y[i,j], 
    # relax overlap constraint for first entry
    if I[0] == 1:
        prob.solve()
        
        status = prob.status
        if status !=1:
            print(I)
            print(status)
        varsdict = {}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                entry = int(key.split(",_")[0][-1])
                player = str(key.split(",_")[1][1:-2])
                Y[(entry,player)] = value
        return(status,varsdict,Y)
    if I[0]>=2:
        for i in range(1,I[0]):
            prob += lpSum(Y[(i,j)]*x[(I[0],j)] for j in all_player_list) <= cov_ub
            #cov_ub+=0.1
        prob.solve(GLPK())
        status = prob.status
        if prob.status != 1:
            print(I)
            print(status)
        varsdict ={}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                if I[0]>=100:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])+100*int(key.split(",_")[0][-3])
                elif I[0]>=10:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])
                else:
                    entry = int(key.split(",_")[0][-1])
                player = key.split(",_")[1][1:-2]
                Y[(entry,player)] = value
        return(status,varsdict,Y)            

### def sequential_optimization_structure

In [8]:
def sequential_opt(k, cov_ub, optimization_form):
    '''
    k_entry: number of entry we want
    result: a dataframe representing result
    cov_ub: a list for pairwise covariance 
    optimization_form: which formulation of optimization
    '''

    progress = 0
    YY={}
    for i in range(1,k):
        for j in all_player_list:
            YY[(i,j)] = 0
    for i in [[x] for x in range(1,k)]:
        progress+=1
        print(str(progress)+"-th entry started>>>")
        (status,varsdict,YY) = optimization_form (k+1,i,cov_ub,YY)
    print(sum(YY.values()))    
    result_table = {}
    for i in range(1,k):
        result_table[i] = []
    for x,y in YY.items():
        if int(y)==1:
            result_table[x[0]].append(x[1])
    result = pd.DataFrame.from_dict(result_table)
    result.columns = ["entry" +str(i) for i in range(1,k)]
    result
    
    return result.T

## use data from DailyFantasyNerd, bend projection with ROTOGRINDER

In [9]:
skater_data = pd.read_csv("DFN NHL Skaters DK 3_24.csv")
goalies_data = pd.read_csv("DFN NHL Goalies DK 3_24.csv")
print(skater_data.shape)
skater_data = skater_data[skater_data["Proj FP"]>=0]
goalies_data = goalies_data[goalies_data["Proj FP"]>0]

skater_data_ROTO = pd.read_csv("nhl-skater 3_20.csv",names = ["NAME", "SALARY", "TEAM", "POS","OPPO","SCORE1","SC2","SC3"])
goalies_data_ROTO = pd.read_csv("nhl-goalie 3_20.csv",names = ["NAME", "SALARY", "TEAM", "POS","OPPO","SCORE1","SC2","SC3"])
skater_data_ROTO = skater_data_ROTO[skater_data_ROTO["SCORE1"]>=0]
goalies_data_ROTO = goalies_data_ROTO[goalies_data_ROTO["SCORE1"]>0]

print(skater_data.shape)
print(skater_data_ROTO.shape)

(174, 26)
(174, 26)
(154, 8)


In [11]:

#blend projection
for i in skater_data.index:
    for j in skater_data_ROTO.index:
        if skater_data.at[i, 'Player Name'] == skater_data_ROTO.at[j, 'NAME']:
            #print(skater_data.at[i, 'Proj FP'])
            #print(skater_data_ROTO.at[j, 'SCORE1'])
            skater_data.loc[i, 'Proj FP'] = 1* skater_data.at[i, 'Proj FP']+ 0*skater_data_ROTO.at[j, 'SCORE1']
for i in goalies_data.index:
    for j in goalies_data_ROTO.index:
        if goalies_data.at[i, 'Player Name'] == goalies_data_ROTO.at[j, 'NAME']:
            #print(skater_data.at[i, 'Proj FP'])
            #print(skater_data_ROTO.at[j, 'SCORE1'])
            goalies_data.loc[i, 'Proj FP'] = 1* goalies_data.at[i, 'Proj FP']+ 0*goalies_data_ROTO.at[j, 'SCORE1']
print(skater_data.shape)
print(goalies_data.shape)

(174, 26)
(6, 24)


In [12]:
skater_data = skater_data.rename(index=str, columns={"Player Name": "Last Name", 
                                                     "Pos": "Position",
                                                     "Opp": "Opponent",
                                                     "PP": "Power_Play",
                                                     "Proj FP": "Projection"})
goalies_data = goalies_data.rename(index=str, columns={"Player Name": "Last Name", 
                                                     "Pos": "Position",
                                                     "Opp": "Opponent",
                                                     "PP": "Power_Play",
                                                     "Proj FP": "Projection"})  

skater_data["Position"] = skater_data["Position"].replace('LW', 'W', regex=True)
skater_data["Position"] = skater_data["Position"].replace('RW', 'W', regex=True)
skater_data['Opponent'] = skater_data['Opponent'].str.replace('@', '', regex=True)
goalies_data['Opponent'] = goalies_data['Opponent'].str.replace('@', '', regex=True)

skater_data["Last Name"] = skater_data["Last Name"].replace(' ', '_', regex=True)
goalies_data["Last Name"] = goalies_data["Last Name"].replace(' ', '_', regex=True)
skater_data["Last Name"] = skater_data["Last Name"].replace('-', '_', regex=True)
goalies_data["Last Name"] = goalies_data["Last Name"].replace('-', '_', regex=True)

skater_data_ROTO["NAME"] = skater_data_ROTO["NAME"].replace(' ', '_', regex=True)
goalies_data_ROTO["NAME"] = goalies_data_ROTO["NAME"].replace(' ', '_', regex=True)
skater_data_ROTO["NAME"] = skater_data_ROTO["NAME"].replace('-', '_', regex=True)
goalies_data_ROTO["NAME"] = goalies_data_ROTO["NAME"].replace('-', '_', regex=True)



skater_data = skater_data[skater_data["Projection"]>=0]
#skater_data = skater_data[skater_data["Last Name"].isin (skater_data_ROTO.NAME.unique())]

print(skater_data.shape)

(174, 26)


In [13]:
#data preprocessing
c_set = skater_data[skater_data["Position"]=="C"]
w_set = skater_data[skater_data["Position"]=="W"]
d_set = skater_data[skater_data["Position"]=="D"]

c_list = c_set["Last Name"].unique()
w_list = w_set["Last Name"].unique()
d_list = d_set["Last Name"].unique()
g_list = goalies_data["Last Name"].unique()
#skater
skater_list = skater_data["Last Name"].unique()
all_player_list = list(set(list(g_list) + list(skater_list)))
#team
team_list = skater_data["Team"].unique()
#line
line_list = skater_data.Line.unique()
line_list = ['L1', 'L2', 'D1', 'D2', 'L3', 'L4','D3']
#we do not want defenseman who is not from first PPL
non_first_powerline_list = (skater_data[ (skater_data["Power_Play"]!="P1") & (skater_data["Position"]=="D") ]
                            ["Last Name"].unique())
#def mu: each player has a projection score
mu={}
for index,row in skater_data.iterrows():
    mu[row["Last Name"]] = row.Projection
for index,row in goalies_data.iterrows():
    mu[row["Last Name"]] = row.Projection
#def salary
salary_dict = {}
for index,row in skater_data.iterrows():
    salary_dict[row["Last Name"]] = row.Salary
for index,row in goalies_data.iterrows():
    salary_dict[row["Last Name"]] = row.Salary
#def team_dict
team_dict = {}
for team in team_list:
    team_dict[team] = []
for index, row in skater_data.iterrows():
    team_dict[row.Team].append(row["Last Name"])
    team_dict[row.Team] = list(set( team_dict[row.Team]))
for index, row in goalies_data.iterrows():
    team_dict[row.Team].append(row["Last Name"])
    team_dict[row.Team] = list(set( team_dict[row.Team]))
#def goalie's opponent set
g_opponent_dict = {}
for x in g_list:
    g_opponent_dict[x]=[]
for index, row in goalies_data.iterrows():
    g_opponent_dict[row["Last Name"]] = row["Opponent"]
# def team dict {team1:[player1..]}
skater_team_dict = {}
for x in team_list:
    skater_team_dict[x] = []
for index,row in skater_data.iterrows():
    skater_team_dict[row.Team].append(row["Last Name"])
    skater_team_dict[row.Team] = list(set(skater_team_dict[row.Team] ))
# def line dict
line_dict = {}
for x in line_list:
    line_dict[x] = []
for index,row in skater_data.iterrows():
    if str(row.Line) != "nan":
        line_dict[row.Line].append(row["Last Name"])
        line_dict[row.Line] = list(set(line_dict[row.Line]))
###? team_line_dict # this only has skater; use it for line stacking
team_line_dict = {}
for x in team_list:
    for y in line_list:
        team_line_dict[(x,y)] = []
for index,row in skater_data.iterrows():
    if str(row.Line) != "nan":
        team_line_dict[(row.Team, row.Line)].append(row["Last Name"])
        team_line_dict[(row.Team, row.Line)] = list(set(team_line_dict[(row.Team, row.Line)]))


In [14]:
skater_data.shape

(174, 26)

## output automation
#### parse output to dk's template format

In [15]:
template_id_df = pd.read_csv("DKSalaries (3).csv")
template_id_df["Unnamed: 12"] = template_id_df["Unnamed: 12"].str.replace(' ', '_', regex=True)
template_id_df["Unnamed: 12"] = template_id_df["Unnamed: 12"].str.replace('-', '_', regex=True)
player_id_dict = {}
for index,row in template_id_df.iterrows():
    player_id_dict[row["Unnamed: 12"]] = row["Unnamed: 13"]
player_score_dict = {}
for index,row in skater_data.iterrows():
    player_score_dict[row["Last Name"]] = row["Actual FP"]
for index,row in goalies_data.iterrows():
    player_score_dict[row["Last Name"]] = row["Actual FP"]

In [45]:
K=200
%time out_put = sequential_opt(K,6, optimization_form = optimization_form_4) # glpk

1-th entry started>>>
2-th entry started>>>
3-th entry started>>>
4-th entry started>>>
5-th entry started>>>
6-th entry started>>>
7-th entry started>>>
8-th entry started>>>
9-th entry started>>>
10-th entry started>>>
11-th entry started>>>
12-th entry started>>>
13-th entry started>>>
14-th entry started>>>
15-th entry started>>>
16-th entry started>>>
17-th entry started>>>
18-th entry started>>>
19-th entry started>>>
20-th entry started>>>
21-th entry started>>>
22-th entry started>>>
23-th entry started>>>
24-th entry started>>>
25-th entry started>>>
26-th entry started>>>
27-th entry started>>>
28-th entry started>>>
29-th entry started>>>
30-th entry started>>>
31-th entry started>>>
32-th entry started>>>
33-th entry started>>>
34-th entry started>>>
35-th entry started>>>
36-th entry started>>>
37-th entry started>>>
38-th entry started>>>
39-th entry started>>>
40-th entry started>>>
41-th entry started>>>
42-th entry started>>>
43-th entry started>>>
44-th entry started>

In [46]:

DK_df = []
for i in range(K-1):
    out_c=[]
    out_d=[]
    out_w=[]
    out_g=[]
    row = out_put.iloc[i]
    for player in row:
        if player in c_list:
            out_c.append(player) 
        elif player in d_list:
            out_d.append(player)
        elif player in w_list:
            out_w.append(player)
        elif player in g_list:
            out_g.append(player)

    if len(out_c)==3:
        out_u = [out_c[0]] 
        out_c = out_c[1:]
    if len(out_w)==4:
        out_u = [out_w[0]]
        out_w = out_w[1:]
    if len(out_d)==3:
        out_u = [out_d[0]] 
        out_d = out_d[1:]
    dk_out_row = out_c + out_w + out_d + out_g + out_u
    DK_df.append(dk_out_row)
    
DK_df = pd.DataFrame(DK_df)     


In [47]:
DK_df.replace(player_id_dict,inplace=True)
DK_df.columns = ["C","C","W","W","W","D","D","G","UTIL"]
DK_df.to_csv("dk_result.csv",index = False)

In [54]:
ex_list = []
for index,row in out_put.iterrows():
    score = 0
    for x in row:
        try:
            score+=player_score_dict[x]
        except:
            pass
        
    if score >=39:
        ex_list.append(score)
        print(index)
        print(score)

entry112
39.6


In [52]:
import statistics
print(statistics.median(ex_list))
print(statistics.mean(ex_list))

29.0
28.951256281407037


In [ ]:
line_vis = out_put.iloc[98]
print(line_vis)


In [57]:
skater_data[skater_data["Last Name"].isin(line_vis)]

,Last Name,Likes,Inj,Position,Salary,Team,Opponent,Rest,Line,Power_Play,...,S SOG,L5 FP,L40 FP,S FP,Floor FP,Ceil FP,Projection,Proj Val,Actual FP,Actual Val
1,Nikita_Kucherov,5.0,NaN,W,7500,TB,WAS,1.0,L1,P1,...,3.0,6.1,5.9,5.4,1.8,8.9,4.70,0.6,10.0,1.3
5,Steven_Stamkos,2.0,NaN,C,7200,TB,WAS,1.0,L2,P1,...,2.8,2.3,4.6,4.4,1.1,7.6,3.70,0.6,9.5,1.3
10,Bo_Horvat,2.0,NaN,C,6600,VAN,OTT,1.0,L1,P1,...,2.9,4.3,3.6,3.7,1.4,6.0,3.22,0.5,9.5,1.4
15,Victor_Hedman,3.0,NaN,D,6100,TB,WAS,1.0,D1,P1,...,2.6,4.4,3.8,3.7,1.6,5.7,3.31,0.6,6.5,1.1
53,Rasmus_Dahlin,1.0,NaN,D,4200,BUF,TOR,2.0,D2,P1,...,2.2,2.3,2.9,2.6,0.8,4.5,2.44,0.6,3.0,0.7
60,Ondrej_Palat,2.0,NaN,W,4000,TB,WAS,1.0,L2,P1,...,1.7,2.6,2.4,2.4,0.5,4.2,2.27,0.7,2.5,0.6
98,Tanner_Pearson,1.0,NaN,W,3300,VAN,OTT,1.0,L1,P2,...,1.7,2.2,1.9,1.8,0.0,3.5,1.75,0.8,10.0,3.0
123,Loui_Eriksson,1.0,NaN,W,2900,VAN,OTT,1.0,L1,P2,...,1.3,0.7,1.3,1.6,-0.3,3.4,1.98,0.8,10.5,3.6


In [58]:
sum(out_put.iloc[98].replace(player_score_dict))

71.3

In [59]:
out_put

,0,1,2,3,4,5,6,7,8
entry1,Alex_Ovechkin,Garret_Sparks,John_Carlson,Trevor_Moore,Victor_Hedman,Nikita_Kucherov,Connor_Brown,Brayden_Point,Nic_Petan
entry2,John_Tavares,Alex_Ovechkin,John_Carlson,Trevor_Moore,Connor_Hellebuyck,Morgan_Rielly,Connor_Brown,Mitchell_Marner,Nic_Petan
entry3,Jeff_Skinner,Jacob_Markstrom,Loui_Eriksson,Alex_Ovechkin,John_Carlson,Evgeny_Kuznetsov,Tanner_Pearson,Rasmus_Dahlin,Bo_Horvat
entry4,Auston_Matthews,Alex_Ovechkin,Garret_Sparks,Trevor_Moore,Victor_Hedman,Evgeny_Kuznetsov,Morgan_Rielly,Connor_Brown,Nic_Petan
entry5,Tom_Wilson,Auston_Matthews,Anders_Nilsson,Alex_Ovechkin,John_Carlson,Trevor_Moore,Evgeny_Kuznetsov,Morgan_Rielly,Connor_Brown
entry6,Cedric_Paquette,Adam_Erne,Alex_Ovechkin,John_Carlson,Yanni_Gourde,Victor_Hedman,Evgeny_Kuznetsov,Connor_Hellebuyck,Nikita_Kucherov
entry7,Jeff_Skinner,Auston_Matthews,Jack_Eichel,Trevor_Moore,Connor_Hellebuyck,Rasmus_Dahlin,Morgan_Rielly,Connor_Brown,Sam_Reinhart
entry8,Tyler_Motte,Jacob_Markstrom,Jay_Beagle,Alex_Ovechkin,John_Carlson,Victor_Hedman,Evgeny_Kuznetsov,Nikita_Kucherov,Tim_Schaller
entry9,Jeff_Skinner,Jack_Eichel,Trevor_Moore,Victor_Hedman,Nikita_Kucherov,Morgan_Rielly,Connor_Brown,Andrei_Vasilevskiy,Nic_Petan
entry10,Jeff_Skinner,Kyle_Okposo,Jack_Eichel,Alex_Ovechkin,Evgeny_Kuznetsov,Rasmus_Dahlin,Sam_Reinhart,Cam_Fowler,John_Gibson


In [ ]:
### ADD COV

In [64]:
import pandas as pd
from functools import reduce
team_list_31 = ['CLS','NYR','MIN','VGK','OTT','VAN','BOS','NYI','CGY','WPG','ARI','EDM','DAL','BUF','MON',
 'CAR','NJ','PHI','CHI','STL','NSH','COL','LA','FLA','TOR','SJ','DET','WAS','PIT','ANH','TB']
import glob
import os
import pandas as pd



# the path to your csv file directory
mycsvdir = 'C:\\Users\\wang mark\\Downloads\\DFN Data'

# get all the csv files in that directory (assuming they have the extension .csv)
csvfiles = glob.glob(os.path.join(mycsvdir, '*.csv'))

# loop through the files and read them in with pandas
all_fp_list = []
all_team_list=[]
for team in team_list_31: 
    s_list = [] # a list of a team's all player's score through the season
    for csvfile in csvfiles:
        s_df = pd.read_csv(csvfile) # read data
        s_df["Player Name"] = s_df["Player Name"].replace(' ', '_', regex=True)
        s_df["Player Name"] = s_df["Player Name"].replace('-', '_', regex=True)
        s_df = s_df[s_df["Team"] == team]  # filter on one team
        if team in s_df.Team.unique() and list(s_df.Team).count(team)>=5: #exclude skeptical datapoints
            s_list.append(s_df[["Player Name",'Actual FP']])              
            # merge fp through time
            
    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Player Name'],how='outer'), s_list)
    df_merged = df_merged.set_index('Player Name')
    
    all_fp_list.append(df_merged)
    all_team_list_31.append(df_merged)
    
from fancyimpute import KNN
cov_dict={}
k=0
for df in all_team_list_31:
    for i,r in df.iterrows():
        if sum(r) <=5:
            df.drop(i, inplace=True)
        df=pd.DataFrame(data=KNN(k=3).fit_transform(df), columns=df.columns, index=df.index)
        cov_table = df.T.cov()
    cov_dict[team_list_31[k]] = cov_table
    k+=1

Using TensorFlow backend.


Imputing row 1/35 with 0 missing, elapsed time: 0.003
Imputing row 1/35 with 0 missing, elapsed time: 0.002
Imputing row 1/35 with 0 missing, elapsed time: 0.002
Imputing row 1/35 with 0 missing, elapsed time: 0.002
Imputing row 1/35 with 0 missing, elapsed time: 0.001
Imputing row 1/35 with 0 missing, elapsed time: 0.001
Imputing row 1/35 with 0 missing, elapsed time: 0.001
Imputing row 1/35 with 0 missing, elapsed time: 0.002
Imputing row 1/35 with 0 missing, elapsed time: 0.002
Imputing row 1/35 with 0 missing, elapsed time: 0.001
Imputing row 1/35 with 0 missing, elapsed time: 0.002
Imputing row 1/35 with 0 missing, elapsed time: 0.001
Imputing row 1/35 with 0 missing, elapsed time: 0.001
Imputing row 1/35 with 0 missing, elapsed time: 0.002
Imputing row 1/35 with 0 missing, elapsed time: 0.001
Imputing row 1/35 with 0 missing, elapsed time: 0.002
Imputing row 1/35 with 0 missing, elapsed time: 0.001
Imputing row 1/35 with 0 missing, elapsed time: 0.002
Imputing row 1/35 with 0 mis

C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input 


Imputing row 1/34 with 0 missing, elapsed time: 0.003
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.002
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.002
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.002
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.003
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.001
Imputing row 1/34 with 0 missing, elapsed time: 0.002
Imputing row 1/34 with 0 mi

C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input 

...

C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input 


Imputing row 1/45 with 0 missing, elapsed time: 0.004
Imputing row 1/45 with 0 missing, elapsed time: 0.002
Imputing row 1/45 with 0 missing, elapsed time: 0.002
Imputing row 1/45 with 0 missing, elapsed time: 0.002
Imputing row 1/45 with 0 missing, elapsed time: 0.002
Imputing row 1/45 with 0 missing, elapsed time: 0.002
Imputing row 1/45 with 0 missing, elapsed time: 0.002
Imputing row 1/45 with 0 missing, elapsed time: 0.001
Imputing row 1/45 with 0 missing, elapsed time: 0.002
Imputing row 1/45 with 0 missing, elapsed time: 0.004
Imputing row 1/45 with 0 missing, elapsed time: 0.002
Imputing row 1/45 with 0 missing, elapsed time: 0.002
Imputing row 1/45 with 0 missing, elapsed time: 0.001
Imputing row 1/45 with 0 missing, elapsed time: 0.001
Imputing row 1/44 with 0 missing, elapsed time: 0.003
Imputing row 1/44 with 0 missing, elapsed time: 0.002
Imputing row 1/44 with 0 missing, elapsed time: 0.002
Imputing row 1/44 with 0 missing, elapsed time: 0.003
Imputing row 1/44 with 0 mi

C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input 

Imputing row 1/44 with 0 missing, elapsed time: 0.002
Imputing row 1/44 with 0 missing, elapsed time: 0.001
Imputing row 1/44 with 0 missing, elapsed time: 0.002
Imputing row 1/44 with 0 missing, elapsed time: 0.002
Imputing row 1/44 with 0 missing, elapsed time: 0.002
Imputing row 1/44 with 0 missing, elapsed time: 0.002
Imputing row 1/44 with 0 missing, elapsed time: 0.003
Imputing row 1/44 with 0 missing, elapsed time: 0.002
Imputing row 1/44 with 0 missing, elapsed time: 0.003
Imputing row 1/44 with 0 missing, elapsed time: 0.002
Imputing row 1/44 with 0 missing, elapsed time: 0.002
Imputing row 1/28 with 0 missing, elapsed time: 0.001
Imputing row 1/28 with 0 missing, elapsed time: 0.001
Imputing row 1/28 with 0 missing, elapsed time: 0.002
Imputing row 1/28 with 0 missing, elapsed time: 0.001
Imputing row 1/28 with 0 missing, elapsed time: 0.002
Imputing row 1/28 with 0 missing, elapsed time: 0.002
Imputing row 1/28 with 0 missing, elapsed time: 0.001
Imputing row 1/28 with 0 mis

C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input matrix is not missing any values")
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\fancyimpute\solver.py:58: UserWarning: Input matrix is not missing any values
  warnings.warn("Input 

In [65]:
all_player_cov = {}
for x in all_player_list:
    for y in all_player_list:
        all_player_cov[x,y]=0
for k,v in cov_dict.items():
    num_player = len(v.index)
    for i in range(num_player):
        for j in range(i,num_player):
            all_player_cov[v.index[i],v.index[j]] = v[v.index[i]][v.index[j]]

In [158]:
def optimization_form_cov (k,I:list,cov_ub:float,Y:dict ):
    '''
    I: we solve for ith entry sequentially; for i in I
    cov_ub: uperbound of pairwise cov
    '''
    prob = pulp.LpProblem(("opt"), pulp.LpMaximize)
    
    xx = pulp.LpVariable.dicts("XX",((i, j,jj,t) for i in range(1,k) for (j,jj) in all_player_cov.keys()
                                     for t in current_team_list), 0 , 1,cat='Integer')
    x = pulp.LpVariable.dicts("X",((i, j) for i in range(1,k) for j in all_player_list), 0 , 1,cat='Integer')
    # t[i,l] indicator: for a lineup i, if there are any player comes from team l
    ## this is for constraint |team| >=3 
    t = pulp.LpVariable.dicts("T",((i, l) for i in range(1,k) for l in team_list), 0 , 1,cat='Integer')

    # v[i,l] complete line indicator (a line has 3player from same line)
    v = pulp.LpVariable.dicts("V",((i, team, line) for i in range(1,k) for team in team_list for line in line_list), 0 , 1,cat='Integer')

    # partial line indicator
    w = pulp.LpVariable.dicts("W",((i, team, line) for i in range(1,k) for team in team_list for line in line_list), 0 , 1,cat='Integer')
    #init a table YY to document previously calculated result for 1....i-1 entry    
    
    #obj
    prob += lpSum(x[(I[0],j)]* mu[j] for j in all_player_list) +lpSum(xx[(I[0],j,jj,t)]* all_player_cov[j,jj] 
            for (j,jj) in all_player_cov.keys() for t in current_team_list)
    
    
    
    #constraints
    # sum(salary)<50000 for each lineup
    for i in I:
        for team in team_dict.keys():
            for j in team_dict[team]:
                for jj in team_dict[team]:  # loop through team first instead of brutely j*jj
                    prob += 2*xx[i,j,jj,team] <= x[i,j]+x[i,jj]
    for i in I:
        prob += lpSum(x[(i,j)] * salary_dict[j] for j in all_player_list) <=50000
    # each lineup has 9 player
    for i in I:
        prob += lpSum(x[(i,j)] for j in all_player_list) == 9
    # for a lineup  2<=center<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in c_list) >= 2
        prob += lpSum(x[(i,j)] for j in c_list) <= 3
    # similar for W,D,G
    #  3<=wing<=4
    for i in I:
        prob += lpSum(x[(i,j)] for j in w_list) >= 3
        prob += lpSum(x[(i,j)] for j in w_list) <= 4
    # 2<=Defense<=3
    for i in I:
        prob += lpSum(x[(i,j)] for j in d_list) >= 2
        prob += lpSum(x[(i,j)] for j in d_list) <= 3
    # goalie ==1
    for i in I:
        prob += lpSum(x[(i,j)] for j in g_list) == 1

    #4.2 team constrant : 
    ## players for each lineup must come from exact 3 team
    ### Notice these three only count for 8 skaters  
    for i in I:
        for l in team_list:
            prob += 6*t[(i,l)] >= lpSum(x[(i,j)] for j in team_dict[l])
            prob += t[(i,l)] <= lpSum(x[(i,j)] for j in team_dict[l])
    for i in I:       
        prob += lpSum(t[(i,l)] for l in team_list) == 3 # at least 3 teams in each lineup
                          
    #4.3
    #goalie stacking
    ## force the goalie variable xik to be zero if the lineup has any skater opposing goalie k
    for i in I:    
        for j in g_list:
            prob += lpSum(x[(i,l)] for l in skater_team_dict[g_opponent_dict[j]]) <= 6*(1-x[i,j])  
            #print(j)
            #print(skater_team_dict[g_opponent_dict[j]])
            #print(lpSum(x[(i,l)] for l in skater_team_dict[g_opponent_dict[j]]) <= 6*(1-x[i,j]))

    #line stacking
    ## force each lineup has one complete line, two partial line
    for i in I:
        for team in team_list: 
            for line in line_list:
                prob += 3 * v[(i,team,line)] <= lpSum(x[(i,j)] for j in team_line_dict[(team,line)])  #at least 3 player to build a complete line
        prob += lpSum(v[(i,team,line)] for line in line_list for team in team_list) >= 1 #at least one complet line
        #print(lpSum(v[(i,team,line)] for line in line_list for team in team_list) >= 1)
    ## at least two player from two distinct line
    for i in I:
        for team in team_list: 
            for line in line_list:
                prob += 2 * w[(i,team,line)] <= lpSum(x[(i,j)] for j in team_line_dict[(team,line)])  #at least 3 player to build a complete line
        prob += lpSum(w[(i,team,line)] for line in line_list for team in team_list) >= 2 #at least one complet line
    ## exclude all defense player that are not in team's first power line
    for i in I:
        for j in non_first_powerline_list:
            prob+= x[(i,j)] == 0

    #4.4 overlap
    # from previously calculated entry we could get a table Y[i,j], 
    # relax overlap constraint for first entry
    if I[0] == 1:
        prob.solve(GLPK(options=['--mipgap', '0.1']))
        
        status = prob.status
        if status !=1:
            print(I)
            print(status)
        varsdict = {}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                entry = int(key.split(",_")[0][-1])
                player = str(key.split(",_")[1][1:-2])
                Y[(entry,player)] = value
        return(status,varsdict,Y)
    if I[0]>=2:
        for i in range(1,I[0]):
            prob += lpSum(Y[(i,j)]*x[(I[0],j)] for j in all_player_list) <= cov_ub
            #cov_ub+=0.1
        prob.solve(GLPK(options=['--mipgap', '0.1']))
        status = prob.status
        if prob.status != 1:
            print(I)
            print(status)
        varsdict ={}
        for var in prob.variables():
            varsdict[var.name] = var.varValue
        for key,value in varsdict.items():
            if "X_" in key:
                if I[0]>=100:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])+100*int(key.split(",_")[0][-3])
                elif I[0]>=10:
                    entry = int(key.split(",_")[0][-1])+10*int(key.split(",_")[0][-2])
                else:
                    entry = int(key.split(",_")[0][-1])
                player = key.split(",_")[1][1:-2]
                Y[(entry,player)] = value
        return(status,varsdict,Y)            

In [159]:
K=10
%time out_put = sequential_opt(K,6, optimization_form = optimization_form_cov) # glpk

1-th entry started>>>


C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\pulp.py:1670: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solutionTime = -clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\solvers.py:373: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solution_time = clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\solvers.py:394: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solution_time += clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\pulp.py:1672: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be remove

2-th entry started>>>


C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\pulp.py:1670: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solutionTime = -clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\solvers.py:373: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solution_time = clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\solvers.py:394: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solution_time += clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\pulp.py:1672: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be remove

3-th entry started>>>


...

8-th entry started>>>


C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\pulp.py:1670: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solutionTime = -clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\solvers.py:373: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solution_time = clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\solvers.py:394: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solution_time += clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\pulp.py:1672: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be remove

9-th entry started>>>


C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\pulp.py:1670: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solutionTime = -clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\solvers.py:373: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solution_time = clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\solvers.py:394: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  self.solution_time += clock()
C:\Users\wang mark\AppData\Local\Continuum\anaconda3\lib\site-packages\pulp\pulp.py:1672: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be remove

5562
Wall time: 24min 55s
